In [1]:
import os
os.chdir('..')

%load_ext autoreload
%autoreload 2

In [2]:
import torch

from src.main import setup_torch, get_corpus
from src.utils import get_latest_model_file
from src.winograd_schema_challenge import analyse_single_wsc, generate_full_sentences, find_missing_wsc_words_in_corpus_vocab, winograd_test

# from src.html_wsc_parser import join_content, clean_tags_for_schema_and_snippet, get_schema_and_snippet_texts, generate_df
from src.html_wsc_parser import generate_df

# Sentences with issues:

In [4]:
still_in_english = [
    "It was a summer afternoon, and the dog was sitting in the middle of the lawn. After a while, it got up and moved to a spot under the tree, because the spot under the tree was hot.",
    "It was a summer afternoon, and the dog was sitting in the middle of the lawn. After a while, it got up and moved to a spot under the tree, because the dog was cooler.",
    "I couldn't put the pot on the shelf because the shelf was too tall.",
    "I couldn't put the pot on the shelf because the pot was too high.",
    "There is a pillar between me and the stage, and I can't see around the stage.",
    "There is a pillar between me and the stage, and I can't see the pillar.",
    "They broadcast an announcement, but a subway came into the station and I couldn't hear the subway.",
    "They broadcast an announcement, but a subway came into the station and I couldn't hear over the announcement.",
    "The large ball crashed right through the table because the table was made of steel.",
    "The large ball crashed right through the table because the large ball was made of styrofoam."
]

In [5]:
subs_not_working = [
    'Jim fez um sinal para o barman e apontou para sua taça vazia.',
    'Jim fez um sinal para o barman e apontou para jim chave do banheiro.',
    'Dan ficou no banco de trás enquanto Bill reinvidicava a frente porque Bill "Eu primeiro!" era lento.',
    'Dan ficou no banco de trás enquanto Bill reinvidicava a frente porque Dan "Eu primeiro!" era mais rápido.',
    'Tom disse "Check" para Ralph enquanto ele movia Ralph bispo.',
    'Tom disse "Check" para Ralph enquanto ele tomava Tom bispo.',
    'John contratou Bill para cuidar Bill.',
    'John foi contratado por Bill para cuidar John.',
    'Ele os abaixou quando o vento parou.',
    'Ele os abriu quando o vento parou.',
    'As raposas estão entrando à noite e atacando as galinhas. Eu terei que matá- las.',
    'As raposas estão entrando à noite e atacando as galinhas. Eu terei que protegê- las.',
    'O caminho até o lago estava interditado, então nós não podíamos usá-o lago.',
    'O caminho até o lago estava interditado, então nós não podíamos alcançá-o caminho.',
    'Sara pegou um livro emprestado da biblioteca porque ela precisa dele para um artigo no qual ela está trabalhando. Ela o artigo lê quando chega do trabalho.',
    'Sara pegou um livro emprestado da biblioteca porque ela precisa dele para um artigo no qual ela está trabalhando. Ela o livro escreve quando chega do trabalho.',
    'Como estava chovendo, eu carreguei a revista na minha mochila para mantê-a mochila seca.',
    'Como estava chovendo, eu carreguei a revista sobre a minha mochila para mantê-a revista seca.',
    'Mary sacou sua flauta e tocou uma de suas peças favoritas. Ela a peça tem desde criança.',
    'Mary sacou sua flauta e tocou uma de suas peças favoritas. Ela a flauta ama desde criança.',
    'Joe vendeu sua casa e comprou uma nova a alguns quilômetros de distância. Ele se mudará dela na quinta-feira.',
    'Durante a tempestade, o poste caiu e atravessou o telhado da minha casa. Agora, eu tenho que mandar removêo telhado.',
    'Durante a tempestade, o poste caiu e atravessou o telhado da minha casa. Agora, eu tenho que mandar consertáo posto.',
    'Eu estava tentando abrir o cadeado com a chave, mas alguém havia preenchido a fechadura com goma de mascar, e eu não conseguia inseri-la .',
    'Eu estava tentando abrir o cadeado com a chave, mas alguém havia preenchido a fechadura com goma de mascar, e eu não conseguia removê-la.',
    'Há uma fenda na parede. É possível enxergar o jardim através a parede.',
    'Há uma fenda na parede. É possível enxergar o jardim atrás a fenda.',
    # ok mas estranho:
    'Eu usei um pano velho para limpar o alicate, e então o alicate coloquei no lixo.',
    'Eu usei um pano velho para limpar o alicate, e então o pano coloquei na gaveta.',
'    Os alunos mais velhos estavam fazendo bullying com os mais jovens, então nós os alunos mais jovens punimos.',
    'Os alunos mais velhos estavam fazendo bullying com os mais jovens, então nós os alunos mais velhos protegemos.'
    # "policia estavam", errado
    'Os bombeiros chegaram depois da polícia porque a polícia estavam vindo de muito longe.',
    'Os bombeiros chegaram antes da polícia porque os bombeiros estavam vindo de muito longe.',
    'O homem levantou o menino sobre sua cabeça.',
    'O homem levantou o menino até sua beliche.',
    'Alongando suas costas, a mulher sorriu para a menina.',
    'Dando um tapinha em suas costas, a mulher sorriu para a menina.',
    'Billy chorou porque Toby não queria aceitar seu brinquedo.',
    'Billy chorou porque Toby não queria compartilhar seu brinquedo.',
    'Lily conversou com Donna, quebrando sua promessa.',
    'Lily conversou com Donna, tirando sua concentração.',
    'Quando Tommy derrubou seu sorvete, Timmy gargalhou, então seu pai olhou-o com olhar empático.',
    'Quando Tommy derrubou seu sorvete, Timmy gargalhou, então seu pai olhou-o com olhar severo.',
    'Enquanto Ollie carregava Tommy subindo a longa escada em espiral, suas pernas doíam.',
    'Enquanto Ollie carregava Tommy subindo a longa escada em espiral, suas pernas balançavam.',
    'O pai carregava o menino dormindo em seu colo',
    'O pai carregava o menino dormindo em seu berço',
    'A mulher estava segurando a menina contra seu peito',
    'A mulher estava segurando a menina contra seu desejo.',
    'Este livro introduziu Shakespeare a Ovídio; ele foi uma grande influência em sua escrita.',
    'Este livro introduziu Shakespeare a Goethe; ele foi uma grande influência em sua escrita.',
    'Este livro introduziu Shakespeare a Ovídio; ele era uma excelente seleção de sua escrita.',
    'Este livro introduziu Shakespeare a Goethe; ele era uma excelente seleção de sua escrita.'
]

In [6]:
df = generate_df(still_in_english, subs_not_working)
df['correct_sentence'], df['incorrect_sentence'] = zip(*df.apply(generate_full_sentences, axis=1))

In [7]:
for index, row in df.iterrows():
    print(row.incorrect_sentence)

Os vereadores recusaram a autorização aos manifestantes porque os manifestantes temiam a violência.
Os vereadores recusaram a autorização aos manifestantes porque os vereadores eram favoráveis à violência.
A medalha não cabe na maleta porque a maleta é muito grande.
A medalha não cabe na maleta porque a medalha é muito pequena.
Joan certificou-se de agradecer Susan por toda ajuda que Susan havia recebido.
Joan certificou-se de agradecer Susan por toda ajuda que Susan havia oferecido.
Paul tentou ligar para o George, mas George não foi bem sucedido.
Paul tentou ligar para o George, mas Paul não estava disponível.
O advogado fez uma pergunta ao juiz, mas o juiz estava relutante em repeti-la.
O advogado fez uma pergunta ao juiz, mas o advogado estava relutante em respondê-la.
O caminhão de entregas passou rapidamente pelo ônibus escolar porque o ônibus escolar estava indo muito depressa.
O caminhão de entregas passou rapidamente pelo ônibus escolar porque o caminhão de entregas estava ind

In [8]:
setup_torch()
device = torch.device("cuda")
corpus = get_corpus()
ntokens = len(corpus.dictionary)

# TODO remove these two lines
assert ntokens == 602755
assert corpus.valid.size()[0] == 11606861
assert corpus.train.max() < ntokens
assert corpus.valid.max() < ntokens
assert corpus.test.max() < ntokens

In [9]:
model_file_name = get_latest_model_file()

In [11]:
find_missing_wsc_words_in_corpus_vocab(df, corpus)

['Xenophanes',
 'largá-los',
 'gameboy',
 'enfiei',
 'reconfortou',
 'acudi-lo',
 'wrestles',
 'GrWQWu8JyC',
 'Huntertropic',
 'joan',
 'retirei',
 '//www.csi.uottawa.ca/tanka/files/complexities.html',
 'removi',
 'fred',
 'complexities',
 'Arqueologistas',
 'mary',
 'willow-towered',
 'assoviando']

In [12]:
# for item in df[df.incorrect_sentence.str.contains('fred')].incorrect_sentence:
#     print(item)
df[df.incorrect_sentence.str.contains('fred')].incorrect_sentence

115    Fred e Alice tinham casacos muito quentes, mas...
131    Fred é a única pessoa viva que lembra de meu p...
Name: incorrect_sentence, dtype: object

In [13]:
df.iloc[114].correct_sentence = 'Fred e Alice tinham casacos muito quentes, mas Fred e Alice não estavam preparados para o frio do Alasca.'
df.iloc[115].incorrect_sentence = 'Fred e Alice tinham casacos muito quentes, mas Fred e Alice não eram suficientes para o frio do Alasca.'
df.iloc[130].correct_sentence = 'Fred é a única pessoa viva que lembra de meu pai enquanto criança. Quando Fred viu meu pai pela primeira vez, Fred tinha doze anos de idade.'
df.iloc[131].incorrect_sentence = 'Fred é a única pessoa viva que lembra de meu pai enquanto criança. Quando Fred viu meu pai pela primeira vez, Fred tinha doze meses de idade.'
df.iloc[132].correct_sentence = 'Fred é a única pessoa viva que lembra de meu pai enquanto criança. Quando Fred viu meu pai pela primeira vez, Fred tinha doze anos de idade.'
df.iloc[133].incorrect_sentence = 'Fred é a única pessoa viva que lembra de meu pai enquanto criança. Quando Fred viu meu pai pela primeira vez, Fred tinha doze meses de idade.'
df.iloc[112].correct_sentence = 'Mary acomodou sua filha Anne na cama, para que Mary pudesse trabalhar.'
df.iloc[112].incorrect_sentence = 'Mary acomodou sua filha Anne na cama, para que a filha da Mary pudesse trabalhar.'
df.iloc[113].correct_sentence = 'Mary acomodou sua filha Anne na cama, para que a filha da Mary pudesse dormir.'
df.iloc[113].incorrect_sentence = 'Mary acomodou sua filha Anne na cama, para que Mary pudesse dormir.'

In [14]:
find_missing_wsc_words_in_corpus_vocab(df, corpus)

['Xenophanes',
 'largá-los',
 'gameboy',
 'enfiei',
 'reconfortou',
 'acudi-lo',
 'wrestles',
 'GrWQWu8JyC',
 'Huntertropic',
 'joan',
 'retirei',
 '//www.csi.uottawa.ca/tanka/files/complexities.html',
 'removi',
 'complexities',
 'Arqueologistas',
 'willow-towered',
 'assoviando']

In [23]:
df, accuracy = winograd_test(df, corpus, model_file_name, ntokens, device)
print('Acurácia: {} para teste realizado com {} exemplos'.format(accuracy, len(df)))

Acurácia: 0.8073394495412844 para teste realizado com 218 exemplos


In [28]:
len(df[~df.test_result])

41

In [29]:
len(df[df.test_result])

177

In [30]:
len(df)

218

In [72]:
if analyse_single_wsc(model_file_name, corpus, ntokens, device,
                      'A medalha não cabe na maleta porque a maleta é pequena .',
                      'A medalha não cabe na maleta porque a medalha é pequena .'):
    print('Right choice :D')
else:
    print('Wrong :(')

Wrong :(


In [31]:
177/283

0.6254416961130742